In [9]:
import json
from datetime import datetime

import requests

# NOAA API Configuration
USER_AGENT = "DailyScarlett (https://github.com/Breza/DailyScarlett)"
POINT_URL = "https://api.weather.gov/points/{lat},{lon}"
HEADERS = {"User-Agent": USER_AGENT}


def get_forecast_data(lat, lon):
    """
    Retrieve structured forecast and alerts for given latitude and longitude.

    Args:
        lat (float): The latitude for the forecast.
        lon (float): The longitude for the forecast.

    Returns:
        dict: A dictionary containing forecast and alerts data, or an error message.
    """
    try:
        # Use a session object for connection pooling and default headers
        with requests.Session() as session:
            session.headers.update(HEADERS)

            # Get NOAA location from coordinates
            point_response = session.get(
                POINT_URL.format(lat=lat, lon=lon),
                timeout=10
            )
            point_response.raise_for_status()
            point_data = point_response.json().get("properties", {})

            # Extract forecast and alerts URLs
            forecast_url = point_data.get("forecast")
            alerts_url = f"https://api.weather.gov/alerts/active?point={lat},{lon}"

            if not forecast_url:
                return {"error": "Could not retrieve forecast URL from the API."}

            # Get 7-day forecast
            forecast_response = session.get(forecast_url, timeout=10)
            forecast_response.raise_for_status()
            forecast_data = forecast_response.json()

            # Get active alerts
            alerts_response = session.get(alerts_url, timeout=10)
            alerts_response.raise_for_status()
            alerts_data = alerts_response.json()

            return {
                "forecast": process_forecast(forecast_data),
                "alerts": process_alerts(alerts_data)
            }

    except requests.exceptions.RequestException as e:
        return {"error": f"API request failed: {e}"}
    except (KeyError, json.JSONDecodeError, AttributeError) as e:
        return {"error": f"Data processing error: {e}"}


def process_forecast(forecast_data):
    """
    Extract today's and tomorrow's forecast from the raw forecast data.

    Args:
        forecast_data (dict): The JSON response from the forecast API endpoint.

    Returns:
        dict: A dictionary with today's and tomorrow's structured forecast.
    """
    periods = forecast_data.get("properties", {}).get("periods", [])
    if len(periods) < 4:
        return {"error": "Insufficient forecast data to process today and tomorrow."}

    # Helper function to extract data for a given period
    def get_period_data(day_period, night_period):
        return {
            "high": day_period.get("temperature"),
            "low": night_period.get("temperature"),
            "day_condition": day_period.get("shortForecast"),
            "night_condition": night_period.get("shortForecast")
        }

    return {
        "today": get_period_data(periods[0], periods[1]),
        "tomorrow": get_period_data(periods[2], periods[3])
    }


def process_alerts(alerts_data):
    """
    Extract and simplify active weather alerts, avoiding duplicates.

    This function is designed to be portable across different Python versions
    and operating systems (Windows, Linux).

    Args:
        alerts_data (dict): The JSON response from the active alerts API endpoint.

    Returns:
        list: A list of simplified alert dictionaries, limited to the first two unique events.
    """
    unique_events = set()
    processed_alerts = []

    for alert in alerts_data.get("features", []):
        props = alert.get("properties", {})
        event = props.get("event")

        if event and event not in unique_events:
            unique_events.add(event)

            expires_iso = props.get("expires")
            expires_time = "N/A"

            if expires_iso:
                try:
                    # Portability fix for older Python versions (pre-3.11) that
                    # cannot parse timezone offsets with a colon (e.g., +00:00).
                    # We manually remove the colon from the timezone offset.
                    if ":" == expires_iso[-3]:
                        expires_iso = expires_iso[:-3] + expires_iso[-2:]

                    dt = datetime.fromisoformat(expires_iso)

                    # Portability fix for Windows: strftime("%-I") is not supported.
                    # Use the original, more portable method for 12-hour format.
                    hour = dt.strftime("%I").lstrip("0")
                    if not hour:  # Handles midnight case where lstrip leaves an empty string
                        hour = "12"
                    expires_time = f"{hour}:{dt:%M} {dt:%p}"

                except (ValueError, TypeError):
                    expires_time = "Invalid time format"

            processed_alerts.append({
                "severity": props.get("severity"),
                "event": event,
                "expires": expires_time
            })

            if len(processed_alerts) >= 2:
                break

    return processed_alerts

In [10]:
if __name__ == "__main__":
    # My house's coordinates
    LATITUDE = 38.904069
    LONGITUDE = -76.936207

    weather_data = get_forecast_data(LATITUDE, LONGITUDE)

    try:
        with open("daily_weather.json", "w") as f:
            json.dump(weather_data, f, indent=2)
        print("Weather data saved to daily_weather.json")
    except IOError as e:
        print(f"Error writing to file: {e}")

Weather data saved to daily_weather.json


In [16]:
weather_data['weekday'] = datetime.now().strftime('%A')

In [17]:
weather_data

{'forecast': {'today': {'high': 95,
   'low': 72,
   'day_condition': 'Mostly Sunny then Scattered Showers And Thunderstorms',
   'night_condition': 'Scattered Showers And Thunderstorms'},
  'tomorrow': {'high': 81,
   'low': 73,
   'day_condition': 'Showers And Thunderstorms Likely',
   'night_condition': 'Showers And Thunderstorms Likely then Chance Showers And Thunderstorms'}},
 'alerts': [{'severity': 'Severe',
   'event': 'Flood Watch',
   'expires': '7:30 PM'},
  {'severity': 'Moderate', 'event': 'Heat Advisory', 'expires': '6:45 PM'}],
 'weekday': 'Thursday'}

In [43]:
from printer import print_receipt

In [44]:
print_receipt("Hello Andreas! I have a receipt printer now.")

Print job sent.
Printer connection closed.
